In [ ]:
#@title ###### Licensed to the Apache Software Foundation (ASF), Version 2.0 (the "License")

# Licensed to the Apache Software Foundation (ASF) under one
# or more contributor license agreements. See the NOTICE file
# distributed with this work for additional information
# regarding copyright ownership. The ASF licenses this file
# to you under the Apache License, Version 2.0 (the
# "License"); you may not use this file except in compliance
# with the License. You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing,
# software distributed under the License is distributed on an
# "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
# KIND, either express or implied. See the License for the
# specific language governing permissions and limitations
# under the License.

# RunInference

<button>
  <a href="https://beam.apache.org/documentation/sdks/python-machine-learning/">
    <img src="https://beam.apache.org/images/favicon.ico" alt="Open the docs" height="16"/>
    Beam RunInference
  </a>
</button>

In this notebook, we walk through the use of the RunInference transform.
The transform and its accompanying [ModelHandler](https://beam.apache.org/releases/pydoc/current/apache_beam.ml.inference.base.html#apache_beam.ml.inference.base.ModelHandler) classes handle the following tasks:


*   Optimizing loading models from popular frameworks.
*   Batching examples in a scalable fashion.


This notebook illustrates common RunInference patterns such as the following:
*   Generating predictions using both Pytorch and Scikit-learn.
*   Post processing results after RunInference.
*   Inference with multiple models in the same pipeline.

The linear regression models used in these samples are trained on data that correspondes to the 5 and 10 times table; that is,`y = 5x` and `y = 10x` respectively.

### Dependencies

The RunInference library is available in Apache Beam version <b>2.40</b> or later.

Pytorch module is needed to use Pytorch RunInference API. use `pip` to install Pytorch.

In [ ]:
# issue: https://github.com/apache/beam/issues/22218. Becuase of the updates to the Google cloud APIs, Beam SDK from 2.34.0 till 2.40.0 has some dependency conflicts. See the issue for more details.
# Workaround to install the apache beam without getting stuck for long time. Runtime might need to restart after this step.
!pip install google-api-core==1.31.6 --quiet
!pip install google-cloud-pubsub==2.13.1 google-cloud-bigquery-storage==2.13.2 --quiet
!pip install apache-beam[gcp,dataframe] --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-gbq 0.13.3 requires google-cloud-bigquery[bqstorage,pandas]<2.0.0dev,>=1.11.1, but you have google-cloud-bigquery 2.34.4 which is incompatible.


In [ ]:
%pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import argparse
import csv
import json
import os
import torch
from typing import Tuple

import apache_beam as beam
import numpy
from apache_beam.io.gcp.bigquery import ReadFromBigQuery
from apache_beam.ml.inference.base import KeyedModelHandler
from apache_beam.ml.inference.base import PredictionResult
from apache_beam.ml.inference.base import RunInference
from apache_beam.dataframe.convert import to_pcollection
from apache_beam.ml.inference.pytorch_inference import PytorchModelHandlerTensor
from apache_beam.ml.inference.pytorch_inference import PytorchModelHandlerKeyedTensor
from apache_beam.options.pipeline_options import PipelineOptions

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
# Constants
project = "<your-project>"
bucket = "<your-bucket>"

# set the project to avoid warnings.
os.environ['GOOGLE_CLOUD_PROJECT'] = project

save_model_dir_multiply_five = 'five_times_table_torch.pt'
save_model_dir_multiply_ten = 'ten_times_table_torch.pt'

## Create data and Pytorch models for RunInference transform

### Linear regression model in Pytorch.

In [ ]:
class LinearRegression(torch.nn.Module):
    def __init__(self, input_dim=1, output_dim=1):
        super().__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)  
    def forward(self, x):
        out = self.linear(x)
        return out

### Prepare train and test data to train a 5 times model.
* `x` contains values in the range from 0 to 99.
* `y` is a list of 5 * `x`. 
* `value_to_predict` includes values outside of the training data.

In [ ]:
x = numpy.arange(0, 100, dtype=numpy.float32).reshape(-1, 1)
y = (x * 5).reshape(-1, 1)
value_to_predict = numpy.array([20, 40, 60, 90], dtype=numpy.float32).reshape(-1, 1)

### Train the linear regression mode on 5 times data.

In [ ]:
five_times_model = LinearRegression()
optimizer = torch.optim.Adam(five_times_model.parameters())
loss_fn = torch.nn.L1Loss()

"""
Train the five_times_model
"""
epochs = 10000
tensor_x = torch.from_numpy(x)
tensor_y = torch.from_numpy(y)
for epoch in range(epochs):
    y_pred = five_times_model(tensor_x)
    loss = loss_fn(y_pred, tensor_y)
    five_times_model.zero_grad()
    loss.backward()
    optimizer.step()

Save the model using `torch.save()` and verify if the saved model file exists.

In [ ]:
torch.save(five_times_model.state_dict(), save_model_dir_multiply_five)
print(os.path.exists(save_model_dir_multiply_five)) # verify if the model is saved

True


### Prepare train and test data to train a 10 times model.
* `x` contains values in the range from 0 to 99.
* `y` is a list of 10 * `x`. 

In [ ]:
x = numpy.arange(0, 100, dtype=numpy.float32).reshape(-1, 1)
y = (x * 10).reshape(-1, 1)

### Train the linear regression model on 10 times data.

In [ ]:
ten_times_model = LinearRegression()
optimizer = torch.optim.Adam(ten_times_model.parameters())
loss_fn = torch.nn.L1Loss()

epochs = 10000
tensor_x = torch.from_numpy(x)
tensor_y = torch.from_numpy(y)
for epoch in range(epochs):
    y_pred = ten_times_model(tensor_x)
    loss = loss_fn(y_pred, tensor_y)
    ten_times_model.zero_grad()
    loss.backward()
    optimizer.step()

Save the model using `torch.save()`

In [ ]:
torch.save(ten_times_model.state_dict(), save_model_dir_multiply_ten)
print(os.path.exists(save_model_dir_multiply_ten)) # verify if the model is saved

True


# Pattern 1: RunInference for predictions.

### Step 1 - Use RunInference within the pipeline.

1. Create pytorch model handler object by passing required arguments such as `state_dict_path`, `model_class`, `model_params` to the `PytorchModelHandlerTensor` class.
2. Pass the `PytorchModelHandlerTensor` object to the RunInference transform to peform prediction on unkeyed data.

In [ ]:
torch_five_times_model_handler = PytorchModelHandlerTensor(
    state_dict_path=save_model_dir_multiply_five,
    model_class=LinearRegression,
    model_params={'input_dim': 1,
                  'output_dim': 1}
                  )
pipeline = beam.Pipeline()

with pipeline as p:
      (
      p 
      | "ReadInputData" >> beam.Create(value_to_predict)
      | "ConvertNumpyToTensor" >> beam.Map(torch.Tensor)
      | "RunInferenceTorch" >> RunInference(torch_five_times_model_handler)
      | beam.Map(print)
      )

PredictionResult(example=tensor([20.]), inference=tensor([99.9943], grad_fn=<UnbindBackward0>))
PredictionResult(example=tensor([40.]), inference=tensor([199.9889], grad_fn=<UnbindBackward0>))
PredictionResult(example=tensor([60.]), inference=tensor([299.9835], grad_fn=<UnbindBackward0>))
PredictionResult(example=tensor([90.]), inference=tensor([449.9753], grad_fn=<UnbindBackward0>))


# Pattern 2: Post-process RunInference results.
Add a `PredictionProcessor` to the pipeline after `RunInference`. `PredictionProcessor` processes the output of the `RunInference` transform.

In [ ]:
class PredictionProcessor(beam.DoFn):
  """
  A processor to format the output of the RunInference transform.
  """
  def process(
      self,
      element: PredictionResult):
    input_value = element.example
    output_value = element.inference
    yield (f"input is {input_value.item()} output is {output_value.item()}")

pipeline = beam.Pipeline()

with pipeline as p:
    (
    p
    | "ReadInputData" >> beam.Create(value_to_predict)
    | "ConvertNumpyToTensor" >> beam.Map(torch.Tensor)
    | "RunInferenceTorch" >> RunInference(torch_five_times_model_handler)
    | "PostProcessPredictions" >> beam.ParDo(PredictionProcessor())
    | beam.Map(print)
    )

input is 20.0 output is 99.99430084228516
input is 40.0 output is 199.98887634277344
input is 60.0 output is 299.98345947265625
input is 90.0 output is 449.9753112792969


# Pattern 3: Attach a key

## Step 1 - Create a source with attached key.


## Step 2 - Modify model handler and post processor.
* Modify the pipeline to read from sources like CSV files and BigQuery.

In this step we:

* Wrap the `PytorchModelHandlerTensor` object around `KeyedModelHandler` to handle keyed data.
* Add a map transform, which converts a table row into `Tuple[str, float]`.
* Add a map transform which converts `Tuple[str, float]` from  to `Tuple[str, torch.Tensor]`.
* Modify the post inference processor to output results along with the key.

In [ ]:
class PredictionWithKeyProcessor(beam.DoFn):
    def __init__(self):
        beam.DoFn.__init__(self)

    def process(
          self,
          element: Tuple[str, PredictionResult]):
        key = element[0]
        input_value = element[1].example
        output_value = element[1].inference
        yield (f"key: {key}, input: {input_value.item()} output: {output_value.item()}" )

#### Use BigQuery as the source.

Install Google Cloud BigQuery API using `pip`.

In [ ]:
%pip install --upgrade google-cloud-bigquery --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-gbq 0.13.3 requires google-cloud-bigquery[bqstorage,pandas]<2.0.0dev,>=1.11.1, but you have google-cloud-bigquery 3.3.0 which is incompatible.


Create a table in the BigQuery using the snippet below, which has two columns: One holds the key and the second holds the test value. To use BiqQuery, a Google Cloud account with the BigQuery API enabled is required.

In [ ]:
!gcloud config set project $project

Updated property [core/project].


In [ ]:
from google.cloud import bigquery

client = bigquery.Client(project=project)

# Make sure the dataset_id is unique in your project.
dataset_id = '{project}.maths'.format(project=project)
dataset = bigquery.Dataset(dataset_id)

# Modify the location based on your project configuration.
dataset.location = 'US'
dataset = client.create_dataset(dataset, exists_ok=True)

# Table name in the BigQuery dataset.
table_name = 'maths_problems_1'

query = """
    CREATE OR REPLACE TABLE
      {project}.maths.{table} ( key STRING OPTIONS(description="A unique key for the maths problem"),
    value FLOAT64 OPTIONS(description="Our maths problem" ) );
    INSERT INTO maths.{table}
    VALUES
      ("first_question", 105.00),
      ("second_question", 108.00),
      ("third_question", 1000.00),
      ("fourth_question", 1013.00)
""".format(project=project, table=table_name)

create_job = client.query(query)
create_job.result()

Use `BigQuery` as the source in the pipeline to read keyed data.

In [ ]:
pipeline_options = PipelineOptions().from_dictionary({'temp_location':f'{bucket}/tmp',
                                                      })
pipeline = beam.Pipeline(options=pipeline_options)

keyed_torch_five_times_model_handler = KeyedModelHandler(torch_five_times_model_handler)

table_name = 'maths_problems_1'
table_spec = f'{project}:maths.{table_name}'

with pipeline as p:
      (
      p
      | "ReadFromBQ" >> beam.io.ReadFromBigQuery(table=table_spec) 
      | "PreprocessData" >> beam.Map(lambda x: (x['key'], x['value']))
      | "ConvertNumpyToTensor" >> beam.Map(lambda x: (x[0], torch.Tensor([x[1]])))
      | "RunInferenceTorch" >> RunInference(keyed_torch_five_times_model_handler)
      | "PostProcessPredictions" >> beam.ParDo(PredictionWithKeyProcessor())
      | beam.Map(print)
      )

key: first_question, input: 105.0 output: 524.9712524414062
key: second_question, input: 108.0 output: 539.970458984375
key: third_question, input: 1000.0 output: 4999.72802734375
key: fourth_question, input: 1013.0 output: 5064.724609375


### Using CSV file as the source.

Create a CSV file with two columns: one named `key` that holds the keys, and a second named `value` that holds the test values.

In [ ]:
# creates a csv file with the below values.
csv_values = [("first_question", 105.00),
      ("second_question", 108.00),
      ("third_question", 1000.00),
      ("fourth_question", 1013.00)]
input_csv_file = "./maths_problem.csv"

with open(input_csv_file, 'w') as f:
  writer = csv.writer(f)
  writer.writerow(['key', 'value'])
  for row in csv_values:
    writer.writerow(row)

assert os.path.exists(input_csv_file) == True

In [ ]:
pipeline_options = PipelineOptions().from_dictionary({'temp_location':f'{bucket}/tmp',
                                                      })
pipeline = beam.Pipeline(options=pipeline_options)

keyed_torch_five_times_model_handler = KeyedModelHandler(torch_five_times_model_handler)

with pipeline as p:
  df = p | beam.dataframe.io.read_csv(input_csv_file)
  pc = to_pcollection(df)
  (pc
    | "ConvertNumpyToTensor" >> beam.Map(lambda x: (x[0], torch.Tensor([x[1]])))
    | "RunInferenceTorch" >> RunInference(keyed_torch_five_times_model_handler)
    | "PostProcessPredictions" >> beam.ParDo(PredictionWithKeyProcessor())
    | beam.Map(print)
    )

key: first_question, input: 105.0 output: 524.9712524414062
key: second_question, input: 108.0 output: 539.970458984375
key: third_question, input: 1000.0 output: 4999.72802734375
key: fourth_question, input: 1013.0 output: 5064.724609375


# Pattern 4: Inference with multiple models in the same pipeline.

## Inference with multiple models in parallel. 

Create a torch model handler for the 10 times model using `PytorchModelHandlerTensor`.

In [ ]:
torch_ten_times_model_handler = PytorchModelHandlerTensor(state_dict_path=save_model_dir_multiply_ten,
                                        model_class=LinearRegression,
                                        model_params={'input_dim': 1,
                                                      'output_dim': 1}
                                        )
keyed_torch_ten_times_model_handler = KeyedModelHandler(torch_ten_times_model_handler)

In this, the same data is run through two different models: the one that we’ve been using to multiply by 5 
and a new model, which will learn to multiply by 10.

In [ ]:
pipeline_options = PipelineOptions().from_dictionary(
                                      {'temp_location':f'{bucket}/tmp'})

pipeline = beam.Pipeline(options=pipeline_options)

read_from_bq = beam.io.ReadFromBigQuery(table=table_spec)

with pipeline as p:
  multiply_five = (
      p 
      |  read_from_bq
      | "CreateMultiplyFiveTuple" >> beam.Map(lambda x: ('{} {}'.format(x['key'], '* 5'), x['value']))
      | "ConvertNumpyToTensorFiveTuple" >> beam.Map(lambda x: (x[0], torch.Tensor([x[1]])))
      | "RunInferenceTorchFiveTuple" >> RunInference(keyed_torch_ten_times_model_handler)
  )
  multiply_ten = (
      p 
      | read_from_bq 
      | "CreateMultiplyTenTuple" >> beam.Map(lambda x: ('{} {}'.format(x['key'], '* 10'), x['value']))
      | "ConvertNumpyToTensorTenTuple" >> beam.Map(lambda x: (x[0], torch.Tensor([x[1]])))
      | "RunInferenceTorchTenTuple" >> RunInference(keyed_torch_ten_times_model_handler)
  )

  inference_result = ((multiply_five, multiply_ten) | beam.Flatten() 
                                 | beam.ParDo(PredictionWithKeyProcessor()))
  inference_result | beam.Map(print)

key: first_question * 5, input: 105.0 output: 1046.1859130859375
key: second_question * 5, input: 108.0 output: 1075.8590087890625
key: third_question * 5, input: 1000.0 output: 9898.654296875
key: fourth_question * 5, input: 1013.0 output: 10027.2373046875
key: first_question * 10, input: 105.0 output: 1046.1859130859375
key: second_question * 10, input: 108.0 output: 1075.8590087890625
key: third_question * 10, input: 1000.0 output: 9898.654296875
key: fourth_question * 10, input: 1013.0 output: 10027.2373046875


## Inference with multiple models in sequence 

In a sequential pattern, data is sent to one or more models in sequence, 
with the output from each model chaining to the next model.

1. Read the data from BigQuery.
2. Map the data.
3. RunInference with multiply by 5 model.
4. Process the results.
5. RunInference with multiply by 10 model.
6. Process the results.


In [ ]:
def process_interim_inference(element):
    key, prediction_result = element
    input_value = prediction_result.example
    inference = prediction_result.inference
    formatted_input_value = 'original input is `{} {}`'.format(key, input_value)
    return formatted_input_value, inference


pipeline_options = PipelineOptions().from_dictionary(
                                      {'temp_location':f'{bucket}/tmp'})
pipeline = beam.Pipeline(options=pipeline_options)

with pipeline as p:
  multiply_five = (
      p 
      | beam.io.ReadFromBigQuery(table=table_spec) 
      | "CreateMultiplyFiveTuple" >> beam.Map(lambda x: (x['key'], x['value']))
      | "ConvertNumpyToTensorFiveTuple" >> beam.Map(lambda x: (x[0], torch.Tensor([x[1]])))
      | "RunInferenceTorchFiveTuple" >> RunInference(keyed_torch_five_times_model_handler)
  )

  inference_result = (
    multiply_five 
      | "ExtractResult" >> beam.Map(process_interim_inference) 
      | "RunInferenceTorchTenTuple" >> RunInference(keyed_torch_ten_times_model_handler)
      | beam.ParDo(PredictionWithKeyProcessor())
    )
  inference_result | beam.Map(print)

key: original input is `first_question tensor([105.])`, input: 524.9712524414062 output: 5200.13232421875
key: original input is `second_question tensor([108.])`, input: 539.970458984375 output: 5348.490234375
key: original input is `third_question tensor([1000.])`, input: 4999.72802734375 output: 49460.0703125
key: original input is `fourth_question tensor([1013.])`, input: 5064.724609375 output: 50102.953125


# Sklearn implementation of RunInference API.

Here, we showcase the Sklearn implementation of the RunInference API with the unkeyed data and the keyed data.

Sklearn is a build-dependency of Apache Beam. If a different version of sklearn needs to be installed, use `%pip install scikit-learn==<version>`

In [ ]:
import pickle
from sklearn import linear_model

import numpy as np
from apache_beam.ml.inference.sklearn_inference import ModelFileType
from apache_beam.ml.inference.sklearn_inference import SklearnModelHandlerNumpy

## Create the data and the Sklearn model.
In this cell, we perform:
1. Create the data to train the Sklearn linear regression model.
2. Train the linear regression model.
3. Save the Sklearn model using `pickle`.

In [ ]:
# Input data to train the sklearn model.
x = numpy.arange(0, 100, dtype=numpy.float32).reshape(-1, 1)
y = (x * 5).reshape(-1, 1)

regression = linear_model.LinearRegression()
regression.fit(x,y)

sklearn_model_filename = 'sklearn_5x_model.pkl'
with open(sklearn_model_filename, 'wb') as f:
    pickle.dump(regression, f)

### Scikit-learn RunInference pipeline.

1. Define the Sklearn model handler that accepts array_like object as input.
2. Read the data from BigQuery.
3. Use the Sklearn trained model and the Sklearn RunInference transform on unkeyed data.

In [ ]:
# SklearnModelHandlerNumpy accepts only unkeyed examples.
sklearn_model_handler = SklearnModelHandlerNumpy(model_uri=sklearn_model_filename,
                                                 model_file_type=ModelFileType.PICKLE) # Use ModelFileType.JOBLIB if the model is seriazlized using joblib.


pipeline_options = PipelineOptions().from_dictionary(
                                      {'temp_location':f'{bucket}/tmp'})
pipeline = beam.Pipeline(options=pipeline_options)

with pipeline as p:
  (
      p 
      | "ReadFromBQ" >> beam.io.ReadFromBigQuery(table=table_spec)
      | "ExtractInputs" >> beam.Map(lambda x: [x['value']]) 
      | "RunInferenceSklearn" >> RunInference(model_handler=sklearn_model_handler)
      | beam.Map(print)
  )

PredictionResult(example=[105.0], inference=array([525.]))
PredictionResult(example=[108.0], inference=array([540.]))
PredictionResult(example=[1000.0], inference=array([5000.]))
PredictionResult(example=[1013.0], inference=array([5065.]))


### Sklearn RunInference on keyed inputs.
1. Wrap the `SklearnModelHandlerNumpy` object around `KeyedModelHandler` to handle keyed data.
2. Read the data from BigQuery.
3. Use the Sklearn trained model and the Sklearn RunInference transform on a keyed data.

In [ ]:
sklearn_model_handler = SklearnModelHandlerNumpy(model_uri=sklearn_model_filename,
                                                 model_file_type=ModelFileType.PICKLE) # Use ModelFileType.JOBLIB if the model is serialized using joblib.

keyed_sklearn_model_handler = KeyedModelHandler(sklearn_model_handler)

pipeline_options = PipelineOptions().from_dictionary(
                                      {'temp_location':f'{bucket}/tmp'})
pipeline = beam.Pipeline(options=pipeline_options)

with pipeline as p:
  (
  p 
  | "ReadFromBQ" >> beam.io.ReadFromBigQuery(table=table_spec)
  | "ExtractInputs" >> beam.Map(lambda x: (x['key'], [x['value']])) 
  | "RunInferenceSklearn" >> RunInference(model_handler=keyed_sklearn_model_handler)
  | beam.Map(print)
  )

('first_question', PredictionResult(example=[105.0], inference=array([525.])))
('second_question', PredictionResult(example=[108.0], inference=array([540.])))
('third_question', PredictionResult(example=[1000.0], inference=array([5000.])))
('fourth_question', PredictionResult(example=[1013.0], inference=array([5065.])))


# Cross framework transforms in a single pipeline

In this pipeline, RunInference transforms of different frameworks are used in a single pipeline sequentially. 

In the below cells, we perform the following actions:
1. Create `KeyedModelHandler` for Sklearn and Pytorch. 
2. Run inference on Sklearn and perform intermediate processing using `process_interim_inference`.
3. Take the intermediate result from Sklearn RunInference transform and run that through Pytorch RunInference transform.

In [ ]:
def process_interim_inference(element: Tuple[str, PredictionResult]):
    """
    Returns the key and the prediction to the next RunInference transform.
    """
    key, prediction_result = element
    prediction = prediction_result.inference
    return key, prediction

class PredictionProcessor(beam.DoFn):
    def process(self, element: Tuple[str, PredictionResult]):
        key, prediction_result = element
        input_from_upstream = prediction_result.example
        prediction = prediction_result.inference
        yield (key, prediction.item())

In [ ]:
pipeline_options = PipelineOptions().from_dictionary(
                                      {'temp_location':f'{bucket}/tmp'})
pipeline = beam.Pipeline(options=pipeline_options)

read_from_bq = beam.io.ReadFromBigQuery(table=table_spec)
keyed_inputs = "ExtractKeyedInputs" >> beam.Map(lambda x: (x['key'], [x['value']]))

keyed_sklearn_model_handler = KeyedModelHandler(SklearnModelHandlerNumpy(
    model_uri=sklearn_model_filename,
    model_file_type=ModelFileType.PICKLE))

keyed_torch_model_handler = KeyedModelHandler(PytorchModelHandlerTensor(
    state_dict_path=save_model_dir_multiply_ten,
    model_class=LinearRegression,
    model_params={'input_dim': 1,
                  'output_dim': 1}))

with pipeline as p:
  sklearn_inference_result = (
      p
      | read_from_bq
      | keyed_inputs
      | "RunInferenceSklearn" >> RunInference(model_handler=keyed_sklearn_model_handler)
      | "ExtractOutputs" >> beam.Map(process_interim_inference)
  )

  torch_inference_result = (
      sklearn_inference_result
      | "ConvertNumpyToTensorFiveTuple" >> beam.Map(lambda x: (x[0], torch.Tensor([x[1]])))
      | "RunInferenceTorchFiveTuple" >> RunInference(keyed_torch_five_times_model_handler)
      | "ProcessPredictions" >> beam.ParDo(PredictionProcessor())
      | beam.Map(print)
  )


('first_question', 2624.857421875)
('second_question', 2699.853271484375)
('third_question', 24998.642578125)
('fourth_question', 25323.625)
